In [187]:
from pyspark.sql.functions import *

In [188]:
import pyspark as ps


In [189]:
import re

In [190]:
spark = ps.sql.SparkSession.builder \
            .master("local[4]") \
            .appName("df capstone") \
            .getOrCreate()

dframe = spark.read.json('./data/mpd.slice.0-999.json', multiLine=True)

In [191]:
sc = spark.sparkContext

In [192]:
dframe.printSchema()


root
 |-- info: struct (nullable = true)
 |    |-- generated_on: string (nullable = true)
 |    |-- slice: string (nullable = true)
 |    |-- version: string (nullable = true)
 |-- playlists: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- collaborative: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- duration_ms: long (nullable = true)
 |    |    |-- modified_at: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- num_albums: long (nullable = true)
 |    |    |-- num_artists: long (nullable = true)
 |    |    |-- num_edits: long (nullable = true)
 |    |    |-- num_followers: long (nullable = true)
 |    |    |-- num_tracks: long (nullable = true)
 |    |    |-- pid: long (nullable = true)
 |    |    |-- tracks: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- album_name: string (nullable = true)
 |    |    |    |

In [193]:
type(dframe)

pyspark.sql.dataframe.DataFrame

In [194]:
#dframe.select('playlists.name', explode('playlists.pid')).show(5, truncate=80)

In [195]:
#filtered = dframe.select(get_json_object(str(dframe['playlists']), '$.name').alias('name)'))

In [196]:
dframe.show()

+--------------------+--------------------+
|                info|           playlists|
+--------------------+--------------------+
|[2017-12-03 08:41...|[[false,, 1153241...|
+--------------------+--------------------+



In [197]:
dframe.createOrReplaceTempView('play_frame')

In [198]:
preview = spark.sql('''
                    SELECT playlists
                    FROM play_frame
''')

In [199]:
preview.show()

+--------------------+
|           playlists|
+--------------------+
|[[false,, 1153241...|
+--------------------+



In [200]:
type(preview)

pyspark.sql.dataframe.DataFrame

In [201]:
preview.toPandas()

,playlists
0,"[(false, None, 11532414, 1493424000, Throwback..."


In [202]:
playlist = dframe.head()[1]

In [204]:
type(playlist), len(playlist)

(list, 1000)

In [205]:
len(playlist[0])

12

In [206]:
single_playlist = playlist[0]

In [207]:
testdict = dict()

In [208]:
for element in single_playlist: 
    print (type(element), element, '\n')

<class 'str'> false 

<class 'NoneType'> None 

<class 'int'> 11532414 

<class 'int'> 1493424000 

<class 'str'> Throwbacks 

<class 'int'> 47 

<class 'int'> 37 

<class 'int'> 6 

<class 'int'> 1 

<class 'int'> 52 

<class 'int'> 0 

<class 'list'> [Row(album_name='The Cookbook', album_uri='spotify:album:6vV5UrXcfyQD1wu4Qo2I9K', artist_name='Missy Elliott', artist_uri='spotify:artist:2wIVse2owClT7go1WT98tk', duration_ms=226863, pos=0, track_name='Lose Control (feat. Ciara & Fat Man Scoop)', track_uri='spotify:track:0UaMYEvWZi0ZqiDOoHU3YI'), Row(album_name='In The Zone', album_uri='spotify:album:0z7pVBGOD7HCIB7S8eLkLI', artist_name='Britney Spears', artist_uri='spotify:artist:26dSoYclwsYLMAKD3tpOr4', duration_ms=198800, pos=1, track_name='Toxic', track_uri='spotify:track:6I9VzXrHxO9rA9A5euc8Ak'), Row(album_name='Dangerously In Love (Alben für die Ewigkeit)', album_uri='spotify:album:25hVFAxTlDvXbx2X2QkUkE', artist_name='Beyoncé', artist_uri='spotify:artist:6vWDO969PvNqNYHIOW5v0m', d

In [209]:
single_tracklist = single_entry[11]

In [210]:
len(single_tracklist)

52

In [211]:
single_tracklist[0].asDict()

{'album_name': 'The Cookbook',
 'album_uri': 'spotify:album:6vV5UrXcfyQD1wu4Qo2I9K',
 'artist_name': 'Missy Elliott',
 'artist_uri': 'spotify:artist:2wIVse2owClT7go1WT98tk',
 'duration_ms': 226863,
 'pos': 0,
 'track_name': 'Lose Control (feat. Ciara & Fat Man Scoop)',
 'track_uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI'}

In [212]:
type(single_playlist)

pyspark.sql.types.Row

In [213]:
single_playlist.asDict()

{'collaborative': 'false',
 'description': None,
 'duration_ms': 11532414,
 'modified_at': 1493424000,
 'name': 'Throwbacks',
 'num_albums': 47,
 'num_artists': 37,
 'num_edits': 6,
 'num_followers': 1,
 'num_tracks': 52,
 'pid': 0,
 'tracks': [Row(album_name='The Cookbook', album_uri='spotify:album:6vV5UrXcfyQD1wu4Qo2I9K', artist_name='Missy Elliott', artist_uri='spotify:artist:2wIVse2owClT7go1WT98tk', duration_ms=226863, pos=0, track_name='Lose Control (feat. Ciara & Fat Man Scoop)', track_uri='spotify:track:0UaMYEvWZi0ZqiDOoHU3YI'),
  Row(album_name='In The Zone', album_uri='spotify:album:0z7pVBGOD7HCIB7S8eLkLI', artist_name='Britney Spears', artist_uri='spotify:artist:26dSoYclwsYLMAKD3tpOr4', duration_ms=198800, pos=1, track_name='Toxic', track_uri='spotify:track:6I9VzXrHxO9rA9A5euc8Ak'),
  Row(album_name='Dangerously In Love (Alben für die Ewigkeit)', album_uri='spotify:album:25hVFAxTlDvXbx2X2QkUkE', artist_name='Beyoncé', artist_uri='spotify:artist:6vWDO969PvNqNYHIOW5v0m', durati

In [215]:
import pandas as pd

In [216]:
type(single_playlist)

pyspark.sql.types.Row

In [217]:
filtered_columns = single_playlist.__fields__.copy()

In [222]:
# filtered_columns.remove('modified_at')
# filtered_columns.remove('name')
# filtered_columns.remove('tracks')
filtered_columns



['collaborative',
 'description',
 'duration_ms',
 'num_albums',
 'num_artists',
 'num_edits',
 'num_followers',
 'num_tracks',
 'pid']

In [223]:
single_playlist.__fields__

['collaborative',
 'description',
 'duration_ms',
 'modified_at',
 'name',
 'num_albums',
 'num_artists',
 'num_edits',
 'num_followers',
 'num_tracks',
 'pid',
 'tracks']

In [224]:
single_dict = single_playlist.asDict()

In [ ]:
#test_frame = pd.DataFrame([single_dict[filtered_columns[0]], single_dict[filtered_columns[1]], single_dict[filtered_columns[2]], single_dict[filtered_columns[3]],single_dict[filtered_columns[4]], single_dict[filtered_columns[5]], single_dict[filtered_columns[6]], single_dict[filtered_columns[7]], single_dict[filtered_columns[8]]], columns=list(filtered_columns)) 

In [229]:
#test_frame = pd.DataFrame(single_dict[filtered_columns[0]], columns=['collaborative']) 
#single_dict[filtered_columns[2]]

In [227]:
pframe = pd.DataFrame.from_dict([single_dict])

In [228]:
pframe

,collaborative,description,duration_ms,modified_at,name,num_albums,num_artists,num_edits,num_followers,num_tracks,pid,tracks
0,false,None,11532414,1493424000,Throwbacks,47,37,6,1,52,0,"[(The Cookbook, spotify:album:6vV5UrXcfyQD1wu4..."


In [239]:
for entry in playlist:
    pframe.append(pd.DataFrame.from_dict([entry]))

In [240]:
pframe

,collaborative,description,duration_ms,modified_at,name,num_albums,num_artists,num_edits,num_followers,num_tracks,pid,tracks
0,false,None,11532414,1493424000,Throwbacks,47,37,6,1,52,0,"[(The Cookbook, spotify:album:6vV5UrXcfyQD1wu4..."


In [233]:
len(playlist)

1000

In [238]:
#pframe.append(pd.DataFrame.from_dict([playlist[3]])) <-- unexpected output??

In [241]:
pframe_test2 = pd.DataFrame(playlist)

In [243]:
pframe_test2

,0,1,2,3,4,5,6,7,8,9,10,11
0,false,None,11532414,1493424000,Throwbacks,47,37,6,1,52,0,"[(The Cookbook, spotify:album:6vV5UrXcfyQD1wu4..."
1,false,None,11656470,1506556800,Awesome Playlist,23,21,5,1,39,1,"[(Eye Of The Tiger, spotify:album:4PT9VulQaQP6..."
2,false,None,14039958,1505692800,korean,51,31,18,1,64,2,"[(On And On, spotify:album:7hq1c0ZrLiSKxzhoRac..."
3,false,None,28926058,1501027200,mat,107,86,4,1,126,3,"[(French Festival, spotify:album:0T9YCy8TruLD6..."
4,false,None,4335282,1401667200,90s,16,16,7,2,17,4,"[(Mellon Collie and the Infinite Sadness, spot..."
...,...,...,...,...,...,...,...,...,...,...,...,...
995,false,None,9917901,1507852800,old,40,36,8,1,41,995,"[(Walkin' On Sunshine (Re-Recorded Versions), ..."
996,false,None,3699248,1479254400,Daze,17,15,13,1,17,996,"[(PARTYNEXTDOOR 3 (P3), spotify:album:2FXGUAES..."
997,false,None,27538723,1410307200,rap,98,82,63,1,119,997,"[(UP!, spotify:album:4I8VoaNOYQn92D8ejMPGlw, L..."
998,false,None,24950143,1507939200,Country,75,40,37,1,108,998,"[(Hunter Hayes (Encore), spotify:album:5hVo5O3..."
